In [1]:
import pandas as pd
user_goods = pd.read_csv('user_goods.csv')
user_goods.head()

,us_id,spu_id,action_type,date,brand_id,cat_id
0,522945.0,338312,0.0,80,10005188,1012
1,44676.0,338312,0.0,80,10005188,1012
2,320812.0,338312,0.0,38,10005188,1012
3,452731.0,338312,0.0,74,10005188,1012
4,259493.0,338312,0.0,52,10005188,1012


In [2]:
df_cat = pd.read_csv('df_cat.csv')
df_cat

,action_type,cat_id,cat_id_bro_1012,cat_id_bro_1056,cat_id_bro_26965,cat_id_bro_26968,cat_id_bro_26971,cat_id_bro_26974,cat_id_bro_26977,cat_id_bro_26980,...,cat_id_buy_6507,cat_id_buy_680,cat_id_buy_688,cat_id_buy_72269,cat_id_buy_73741,cat_id_buy_7469,cat_id_buy_7470,cat_id_buy_75798,cat_id_buy_75799,cat_id_buy_945
0,1.0,6507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,26980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,73741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,32532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,75798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,1.0,75799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,1.0,50513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# 用户类别购买量
def get_us_cat_data(start_date, end_date, df):
    df_train = df.loc[(df['date'] >= start_date) & (df['date'] <= end_date)] # 获取某一时间段内数据
    df_train = df_train[['us_id', 'cat_id', 'action_type']]
    df_train = pd.merge(df_train, df_cat, how='right', on=['cat_id', 'action_type'])
    df_train = pd.get_dummies(df_train, prefix = ['action_type'], columns = ['action_type'])
    # 用户-类别 点击量（49）,购买量（49），用户对该类别的点击量（action_type_0.0），用户对该类别的购买量（action_type_1.0）
    df_train = df_train.groupby(['us_id', 'cat_id'], as_index = False).sum()
    # 用户对各个类别的 点击量+购买量， 用户的总点击量（action_type_0.0）与总购买量（action_type_1.0）
    df_us = df_train.drop('cat_id', axis=1).groupby(['us_id'], as_index = False).sum()
    print df_us.shape
    for i in range(1,50):
        # 用户-类别购买率
        df_us['us_cat_ratio'+df_us.columns[i][11:]] = df_us[df_us.columns[i+49]]/(df_us[df_us.columns[i]]+df_us[df_us.columns[i+49]])
        print df_us.columns[i]
        # 用户-类别偏好2
        df_us[df_us.columns[i]] = df_us[df_us.columns[i]]/df_us['action_type_0.0']
        print df_us.columns[i+49]
        # 用户-类别偏好1
        df_us[df_us.columns[i+49]] = df_us[df_us.columns[i+49]]/df_us['action_type_1.0']
    df_us_cat = pd.merge(df_train[['us_id','cat_id','action_type_0.0','action_type_1.0']], df_us, how='right', on='us_id').fillna(0)
    return df_us_cat
    # return df_train,df_us_cat
# us_id, cat_id, 用户对该类别的点击量，用户对该类别的购买量
# 用户对各个类别的购买率（49），用户对各个类别的点击量（49），用户的总点击，用户的总购买量
# 用户对各个类别的购买量（49）

In [5]:
a = get_us_cat_data(1,7,user_goods)

(118562, 101)
cat_id_bro_1012
cat_id_buy_1012
cat_id_bro_1056
cat_id_buy_1056
cat_id_bro_26965
cat_id_buy_26965
cat_id_bro_26968
cat_id_buy_26968
cat_id_bro_26971
cat_id_buy_26971
cat_id_bro_26974
cat_id_buy_26974
cat_id_bro_26977
cat_id_buy_26977
cat_id_bro_26980
cat_id_buy_26980
cat_id_bro_26983
cat_id_buy_26983
cat_id_bro_271
cat_id_buy_271
cat_id_bro_27449
cat_id_buy_27449
cat_id_bro_27898
cat_id_buy_27898
cat_id_bro_27971
cat_id_buy_27971
cat_id_bro_28004
cat_id_buy_28004
cat_id_bro_28005
cat_id_buy_28005
cat_id_bro_28006
cat_id_buy_28006
cat_id_bro_28007
cat_id_buy_28007
cat_id_bro_28008
cat_id_buy_28008
cat_id_bro_28009
cat_id_buy_28009
cat_id_bro_297
cat_id_buy_297
cat_id_bro_311
cat_id_buy_311
cat_id_bro_32532
cat_id_buy_32532
cat_id_bro_337
cat_id_buy_337
cat_id_bro_356
cat_id_buy_356
cat_id_bro_368
cat_id_buy_368
cat_id_bro_3755
cat_id_buy_3755
cat_id_bro_403
cat_id_buy_403
cat_id_bro_42067
cat_id_buy_42067
cat_id_bro_44997
cat_id_buy_44997
cat_id_bro_48333
cat_id_buy_48333


In [7]:
a

,us_id,cat_id,action_type_0.0_x,action_type_1.0_x,cat_id_bro_1012,cat_id_bro_1056,cat_id_bro_26965,cat_id_bro_26968,cat_id_bro_26971,cat_id_bro_26974,...,us_cat_ratio6507,us_cat_ratio680,us_cat_ratio688,us_cat_ratio72269,us_cat_ratio73741,us_cat_ratio7469,us_cat_ratio7470,us_cat_ratio75798,us_cat_ratio75799,us_cat_ratio945
0,3.0,271,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,3.0,297,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,3.0,356,3.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,3.0,7470,8.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,3.0,27449,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
5,3.0,73741,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
6,4.0,680,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
7,4.0,28006,16.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
8,4.0,28007,5.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
9,4.0,42067,4.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [17]:
def get_brand_cat_data(start_date, end_date, df):
    df_train = df.loc[(df['date'] >= start_date) & (df['date'] <= end_date)] # 获取某一时间段内数据
    df_train = df_train[['brand_id', 'cat_id', 'action_type']]
    df_train = pd.get_dummies(df_train, prefix = ['action_type'], columns = ['action_type'])
    # 品牌-类别 点击量与购买量
    df_train = df_train.groupby(['brand_id', 'cat_id'], as_index = False).sum()
    # 品牌购买率
    df_train['bra_buy_ratio'] = df_train['action_type_1.0']/(df_train['action_type_0.0']+df_train['action_type_1.0'])
    # 类别 点击量与购买量
    df_cat = df_train.drop(['brand_id','bra_buy_ratio'], axis = 1).groupby(['cat_id'], as_index = False).sum()
    # 类别购买率
    df_cat['cat_buy_ratio'] = df_cat['action_type_1.0']/(df_cat['action_type_0.0']+df_cat['action_type_1.0'])
    df_bra_cat = pd.merge(df_train, df_cat, how = 'right', on = 'cat_id')
    df_bra_cat['bra_cat_buy_n'] = df_bra_cat['action_type_1.0_x']/df_bra_cat['action_type_1.0_y']
    df_bra_cat['bra_cat_bro_n'] = df_bra_cat['action_type_0.0_x']/df_bra_cat['action_type_0.0_y']
    df_bra_cat['bra_cat_buy_r'] = df_bra_cat['bra_buy_ratio']/df_bra_cat['cat_buy_ratio']
    return df_bra_cat.fillna(0)
# brand_id, cat_id, 品牌点击量，品牌购买量，品牌购买率，类别点击量，类别购买量，类别购买率，
# 品牌-类别购买量，品牌-类别点击量，品牌-类别购买率（除数为0时的NaN值填充？）

In [18]:
a = get_brand_cat_data(1,10,user_goods)
a

,brand_id,cat_id,action_type_0.0_x,action_type_1.0_x,bra_buy_ratio,action_type_0.0_y,action_type_1.0_y,cat_buy_ratio,bra_cat_buy_n,bra_cat_bro_n,bra_cat_buy_r
0,10000004,271,7.0,0.0,0.000000,670897.0,2673.0,0.003968,0.000000,0.000010,0.000000
1,10000006,271,1.0,0.0,0.000000,670897.0,2673.0,0.003968,0.000000,0.000001,0.000000
2,10000007,271,85.0,0.0,0.000000,670897.0,2673.0,0.003968,0.000000,0.000127,0.000000
3,10000029,271,1.0,0.0,0.000000,670897.0,2673.0,0.003968,0.000000,0.000001,0.000000
4,10000030,271,392.0,0.0,0.000000,670897.0,2673.0,0.003968,0.000000,0.000584,0.000000
5,10000045,271,179.0,1.0,0.005556,670897.0,2673.0,0.003968,0.000374,0.000267,1.399946
6,10000058,271,323.0,1.0,0.003086,670897.0,2673.0,0.003968,0.000374,0.000481,0.777748
7,10000075,271,1.0,0.0,0.000000,670897.0,2673.0,0.003968,0.000000,0.000001,0.000000
8,10000085,271,20.0,0.0,0.000000,670897.0,2673.0,0.003968,0.000000,0.000030,0.000000
9,10000102,271,8.0,0.0,0.000000,670897.0,2673.0,0.003968,0.000000,0.000012,0.000000


In [19]:
a.isnull().sum()

brand_id             0
cat_id               0
action_type_0.0_x    0
action_type_1.0_x    0
bra_buy_ratio        0
action_type_0.0_y    0
action_type_1.0_y    0
cat_buy_ratio        0
bra_cat_buy_n        0
bra_cat_bro_n        0
bra_cat_buy_r        0
dtype: int64

In [20]:
def get_spu_cat_data(start_date, end_date, df):
    df_train = df.loc[(df['date'] >= start_date) & (df['date'] <= end_date)] # 获取某一时间段内数据
    df_train = df_train[['spu_id', 'cat_id', 'action_type']]
    df_train = pd.get_dummies(df_train, prefix = ['action_type'], columns = ['action_type'])
    # 商品-类别 点击量与购买量
    df_train = df_train.groupby(['spu_id', 'cat_id'], as_index = False).sum()
    # 商品购买率
    df_train['spu_buy_ratio'] = df_train['action_type_1.0']/(df_train['action_type_0.0']+df_train['action_type_1.0'])
    # 类别点击量与购买量
    df_cat = df_train.drop(['spu_id','spu_buy_ratio'], axis = 1).groupby(['cat_id'], as_index = False).sum()
    # 品牌购买率
    df_cat['cat_buy_ratio'] = df_cat['action_type_1.0']/(df_cat['action_type_0.0']+df_cat['action_type_1.0'])
    df_spu_cat = pd.merge(df_train, df_cat, how = 'right', on = 'cat_id')
    df_spu_cat['spu_cat_buy_n'] = df_spu_cat['action_type_1.0_x']/df_spu_cat['action_type_1.0_y']
    df_spu_cat['spu_cat_bro_n'] = df_spu_cat['action_type_0.0_x']/df_spu_cat['action_type_0.0_y']
    df_spu_cat['spu_cat_buy_r'] = df_spu_cat['spu_buy_ratio']/df_spu_cat['cat_buy_ratio']
    return df_spu_cat.fillna(0)
# spu_id, cat_id, 商品点击量，商品购买量，商品购买率，类别点击量，类别购买量，类别购买率，
# 商品-类别购买量，商品-类别点击量，商品-类别购买率（除数为0时的NaN值填充？）

In [26]:
a = get_spu_cat_data(1,10,user_goods)
a

,spu_id,cat_id,action_type_0.0_x,action_type_1.0_x,spu_buy_ratio,action_type_0.0_y,action_type_1.0_y,cat_buy_ratio,spu_cat_buy_n,spu_cat_bro_n,spu_cat_buy_r
0,4,311,14.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,4.140489e-06,0.000000
1,7,311,99.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,2.927917e-05,0.000000
2,10,311,3.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,8.872477e-07,0.000000
3,11,311,14.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,4.140489e-06,0.000000
4,12,311,3.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,8.872477e-07,0.000000
5,31,311,4.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,1.182997e-06,0.000000
6,51,311,34.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,1.005547e-05,0.000000
7,52,311,148.0,1.0,0.006711,3381243.0,13757.0,0.004052,0.000073,4.377089e-05,1.656265
8,56,311,26.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,7.689480e-06,0.000000
9,60,311,7.0,0.0,0.000000,3381243.0,13757.0,0.004052,0.000000,2.070245e-06,0.000000


In [23]:
def get_spu_brand_data(start_date, end_date, df):
    df_train = df.loc[(df['date'] >= start_date) & (df['date'] <= end_date)] # 获取某一时间段内数据
    df_train = df_train[['spu_id', 'brand_id', 'action_type']]
    df_train = pd.get_dummies(df_train, prefix = ['action_type'], columns = ['action_type'])
    # 商品-品牌 点击量与购买量
    df_train = df_train.groupby(['spu_id', 'brand_id'], as_index = False).sum()
    # 商品购买率
    df_train['spu_buy_ratio'] = df_train['action_type_1.0']/(df_train['action_type_0.0']+df_train['action_type_1.0'])
    # 品牌点击量与购买量
    df_brand = df_train.drop(['spu_id','spu_buy_ratio'], axis = 1).groupby(['brand_id'], as_index = False).sum()
    # 品牌购买率
    df_brand['bra_buy_ratio'] = df_brand['action_type_1.0']/(df_brand['action_type_0.0']+df_brand['action_type_1.0'])
    df_spu_bra = pd.merge(df_train, df_brand, how = 'right', on = 'brand_id')
    df_spu_bra['spu_bra_buy_n'] = df_spu_bra['action_type_1.0_x']/df_spu_bra['action_type_1.0_y']
    df_spu_bra['spu_bra_bro_n'] = df_spu_bra['action_type_0.0_x']/df_spu_bra['action_type_0.0_y']
    df_spu_bra['spu_bra_buy_r'] = df_spu_bra['spu_buy_ratio']/df_spu_bra['bra_buy_ratio']
    return df_spu_bra.fillna(0)
# spu_id, brand_id, 商品点击量，商品购买量，商品购买率，品牌点击量，品牌购买量，品牌购买率，
# 商品-品牌购买量，商品-品牌点击量，商品-品牌购买率（除数为0时的NaN值填充？）

In [24]:
a = get_spu_brand_data(1,10,user_goods)
a

,spu_id,brand_id,action_type_0.0_x,action_type_1.0_x,spu_buy_ratio,action_type_0.0_y,action_type_1.0_y,bra_buy_ratio,spu_bra_buy_n,spu_bra_bro_n,spu_bra_buy_r
0,4,10007327,14.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.001342,0.000000
1,10103,10007327,21.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.002013,0.000000
2,15357,10007327,6.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.000575,0.000000
3,26518,10007327,34.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.003260,0.000000
4,33896,10007327,5.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.000479,0.000000
5,36409,10007327,153.0,1.0,0.006494,10431.0,17.0,0.001627,0.058824,0.014668,3.990833
6,37236,10007327,13.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.001246,0.000000
7,37676,10007327,30.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.002876,0.000000
8,39130,10007327,1.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.000096,0.000000
9,44021,10007327,6.0,0.0,0.000000,10431.0,17.0,0.001627,0.000000,0.000575,0.000000


In [25]:
a.isnull().sum()

spu_id               0
brand_id             0
action_type_0.0_x    0
action_type_1.0_x    0
spu_buy_ratio        0
action_type_0.0_y    0
action_type_1.0_y    0
bra_buy_ratio        0
spu_bra_buy_n        0
spu_bra_bro_n        0
spu_bra_buy_r        0
dtype: int64

In [20]:
pd.get_dummies(user_goods['cat_id'], prefix = 'cat_id')

,cat_id_271,cat_id_297,cat_id_311,cat_id_337,cat_id_356,cat_id_368,cat_id_403,cat_id_517,cat_id_609,cat_id_680,...,cat_id_48336,cat_id_48337,cat_id_50513,cat_id_50514,cat_id_50515,cat_id_50516,cat_id_72269,cat_id_73741,cat_id_75798,cat_id_75799
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
a  = set(user_goods['cat_id'])
a

{271,
 297,
 311,
 337,
 356,
 368,
 403,
 517,
 609,
 680,
 688,
 945,
 1012,
 1056,
 3755,
 6507,
 7469,
 7470,
 26965,
 26968,
 26971,
 26974,
 26977,
 26980,
 26983,
 27449,
 27898,
 27971,
 28004,
 28005,
 28006,
 28007,
 28008,
 28009,
 32532,
 42067,
 44997,
 48333,
 48334,
 48336,
 48337,
 50513,
 50514,
 50515,
 50516,
 72269,
 73741,
 75798,
 75799}